In [6]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=5a496ded0a5cf4cfe6af633fdc2269e9a9c8047e851393e0b39b63bc9221287a
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [172]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean as mean, col, isnull, when, count

In [14]:
spark = SparkSession.builder.appName("SparkSQL").master("local[*]").getOrCreate()

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
data  =  spark.read.csv('/content/drive/MyDrive/Datasets/brooklyn_sales_map.csv', header= True, inferSchema = True) 

In [28]:
data.show()

+---+--------+--------------------+-----------------------+---------+-----+----+--------+--------------+--------------------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+------------+----------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+-------------------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+--------------------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------------------+--------+--------+--------+-------+--------+--------+----------

In [86]:
data.describe().show()

+-------+------------------+--------+-----------------+-----------------------+------------------+-----------------+------------------+--------+--------------+--------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------------+------------------+------------------+---------+------------------+-----------------+------------------+-----------------+------------------+------------------+--------+-----------------+------------------+------------------+--------------------+-----------------+------------------+-------------------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+------------------+--------------------+---------+--------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------

## Find the average price of housing sale_price and output a new table, which will contain two columns - sale price and deviation from average value.


In [88]:
data.select(['sale_price']).dtypes

[('sale_price', 'double')]

In [89]:
data.select(['sale_price']).describe().show()

+-------+------------------+
|summary|        sale_price|
+-------+------------------+
|  count|            390883|
|   mean|506754.47769281344|
| stddev|2353964.6642246204|
|    min|               0.0|
|    max|      4.99401179E8|
+-------+------------------+



In [93]:
sale_price = data.select(data.colRegex('sale_price'))
sale_price.orderBy(['sale_price'], ascending=[0, 1]).show()

+------------+
|  sale_price|
+------------+
|4.99401179E8|
|      3.45E8|
|       3.4E8|
|   2.76947E8|
|       2.4E8|
|    2.0502E8|
|     2.025E8|
|   1.85445E8|
|      1.71E8|
|      1.69E8|
|      1.65E8|
|      1.61E8|
|       1.6E8|
|      1.58E8|
|    1.5425E8|
|1.38106368E8|
|   1.35212E8|
|      1.35E8|
|     1.324E8|
|      1.25E8|
+------------+
only showing top 20 rows



In [94]:
df = data.select(
    mean(col('sale_price')).alias('mean')
).collect()

average_price = df[0]['mean']

In [95]:
average_price

506754.47769281344

In [106]:
data.select(col('sale_price'), (data['sale_price'] - average_price).alias('deviation_from_average_value')).orderBy(['sale_price'], ascending=[0, 1]).show()

+------------+----------------------------+
|  sale_price|deviation_from_average_value|
+------------+----------------------------+
|4.99401179E8|        4.9889442452230716E8|
|      3.45E8|        3.4449324552230716E8|
|       3.4E8|        3.3949324552230716E8|
|   2.76947E8|        2.7644024552230716E8|
|       2.4E8|         2.394932455223072E8|
|    2.0502E8|         2.045132455223072E8|
|     2.025E8|         2.019932455223072E8|
|   1.85445E8|         1.849382455223072E8|
|      1.71E8|         1.704932455223072E8|
|      1.69E8|         1.684932455223072E8|
|      1.65E8|         1.644932455223072E8|
|      1.61E8|         1.604932455223072E8|
|       1.6E8|         1.594932455223072E8|
|      1.58E8|         1.574932455223072E8|
|    1.5425E8|         1.537432455223072E8|
|1.38106368E8|         1.375996135223072E8|
|   1.35212E8|         1.347052455223072E8|
|      1.35E8|         1.344932455223072E8|
|     1.324E8|        1.3189324552230719E8|
|      1.25E8|        1.24493245

## Find average price of housing sale_price for every borough


In [99]:
data.groupBy(['neighborhood']).mean('sale_price').show()

+-------------------+------------------+
|       neighborhood|   avg(sale_price)|
+-------------------+------------------+
|     FLATBUSH-NORTH| 395259.8847659308|
|      CYPRESS HILLS|307765.78343067935|
|        BENSONHURST|429025.58639982785|
|       BOROUGH PARK| 411294.0367964156|
|    WINDSOR TERRACE|480354.99901250825|
|OCEAN PARKWAY-NORTH|  429713.022237665|
|          NAVY YARD| 1240761.168067227|
|            SEAGATE| 313497.1167883212|
|       CLINTON HILL|  641749.259782796|
|   PROSPECT HEIGHTS| 718212.9194644697|
| DOWNTOWN-METROTECH| 1335569.023217247|
|      FLATBUSH-EAST| 317214.9600358557|
|        COBBLE HILL| 991186.6565906839|
| WILLIAMSBURG-SOUTH| 903288.6895322939|
|   BROOKLYN-UNKNOWN| 1218210.196969697|
|       CONEY ISLAND|475380.59604519774|
|      EAST NEW YORK|263002.60926152975|
|            MADISON| 389140.4982742391|
|           BUSHWICK| 405136.1709061467|
|        FORT GREENE| 832763.1297690764|
+-------------------+------------------+
only showing top

## Output the average housing area gross_sqft for every combination of tax_class and year_of_sale.

In [128]:
filtered = data.filter("gross_sqft > 0")
filtered.select(col('tax_class'), col('year_of_sale'), col('gross_sqft')).groupBy(col('tax_class'), col('year_of_sale')).mean('gross_sqft').orderBy(['tax_class', 'year_of_sale'], ascending=[1,1]).show()

+---------+------------+------------------+
|tax_class|year_of_sale|   avg(gross_sqft)|
+---------+------------+------------------+
|        1|        2003| 2459.302546793495|
|        1|        2004|2451.2298075014883|
|        1|        2005|2468.8683642641345|
|        1|        2006|2498.8409980716415|
|        1|        2007|2505.9932594042184|
|        1|        2008|  2504.60383240121|
|        1|        2009|2453.8950603968938|
|        1|        2010| 2457.146494779459|
|        1|        2011| 2458.282728396449|
|        1|        2012|2515.9785049419443|
|        1|        2013| 2471.600475422362|
|        1|        2014| 2465.060696517413|
|        1|        2015|2310.7560419887704|
|        1|        2016|2404.7579152459816|
|        1|        2017|  2264.14225089119|
|        2|        2003| 43119.57868852459|
|        2|        2004| 47128.57685664939|
|        2|        2005|46787.431279620854|
|        2|        2006| 37893.83309143686|
|        2|        2007| 44434.9

## Output a table that contains the number of null values for every column.



In [174]:
data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+---+--------+------------+-----------------------+---------+-----+---+--------+--------------+--------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+----------+---------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+---------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+---------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------+--------+--------+--------+-------+--------+--------+---+-------+---------+------+------+-------+------+-------+------+-